Hier werde ich all die pieces of code reinpacken, die es nicht in den tatsächlichen Endcode geschafft haben. Diese Zeilen werden deutlich weniger kommentiert sein, da sie teils nur sehr kurz Teil der Arbeit waren.

In [1]:
import pandas as pd
import statsmodels.api as sm

# Assume df is your dataframe
# Create a dummy variable for missing 'Mode' values
df['Mode_Missing'] = df['Mode'].isna().astype(int)

# Convert 'Mode' to dummy variables, keeping NaN as is
df_with_dummies = pd.get_dummies(df[['Mode', 'Speed', 'Language', 'Mode_Missing']], columns=['Mode', 'Language'], drop_first=True)

# Prepare the predictors (X) and the response variable (y)
X = df_with_dummies
y = df['Success']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

# Fit the logistic regression model
model = sm.Logit(y, X).fit()

# Print the summary of the regression
print(model.summary())

NameError: name 'df' is not defined